# The openLCA Result API

openLCA 2 provides a harmonized result interface that can be called from
different APIs (Java, JSON-RPC, gRPC, Rest). All these APIs call the same
backend which is implemented in the openLCA kernel. This kernel is independent
from the openLCA user interface and can be integrated as a service into other
applications.

The idea of the result interface is not to provide some ready-to-use charts and
tables but to provide all possible building blocks with which such higher level
result views can be created (charts, tables, upstream trees, Sankey diagrams).
Thus, the result interface has many methods that often look quite similar. This
document tries to explain what these methods do with examples using the [openLCA
Python IPC API v2](https://github.com/GreenDelta/olca-ipc.py). The calculation
is explained using matrix algebra but this does mean that the implementation
uses exactly these formulas when calculating the results.

In order to run the examples in this document, you need to have the olca-ipc.py
module installed and an IPC server in openLCA runnig (`Tools > Developer tools >
IPC Server`). First, make sure that the `olca-ipc` and `olca-schema` modules are
installed. We also use `pandas` and `matplotlib` for visualisations:

In [ ]:
!pip list | grep 'olca\|pandas'

Now, we import the required modules:

In [ ]:
import olca_ipc as ipc
import olca_schema as lca
import olca_schema.results as res
import pandas as pd

In [ ]:
setup = res.CalculationSetup(
    target=lca.Ref(
        model_type="ProductSystem", id="7c328e9b-d8e3-402b-a1ac-95620d021b99"
    ),
    impact_method=lca.Ref(
        model_type="ImpactMethod", id="787c02f1-d1f2-36d6-8e06-2307cc3ebebc"
    ),
)
client = ipc.Client(8080)
result = client.calculate(setup)

In [ ]:
result.wait_until_ready()

In [ ]:
tech_flows = result.get_tech_flows()
pd.DataFrame(
  [(tf.provider.name, tf.flow.name) for tf in tech_flows],
  columns=["Provider", "Flow"]
).head()

In [ ]:
envi_flows = result.get_envi_flows()
pd.DataFrame(
  [(ef.is_input, ef.flow.name, ef.flow.category) for ef in envi_flows],
  columns=["Is input?", "Flow", "Category"]
).head()

In [ ]:
impact_categories = result.get_impact_categories()
pd.DataFrame(
    [i.name for i in impact_categories],
    columns=["Impact category"]
)

## Technosphere flows

### `/total-requirements`

$$
t = diag(A) * diagm(s)
$$

In [ ]:
t = result.get_total_requirements()
pd.DataFrame(
  [(v.tech_flow.flow.name, v.amount, v.tech_flow.flow.ref_unit) for v in t],
  columns=["Tech. flow", "Amount", "Unit"]
).head()